## Importação da planilha BrazilianCities.xlsx 

#### Utilizando as bibliotecas pandas e psycopg2

In [ ]:
# importa bibliotecas
import pandas as pd
import psycopg2
import psycopg2.extras as extras
from psycopg2 import OperationalError, errorcodes, errors
from io import StringIO


In [ ]:
# Variáveis de conexão
host = ''
dbName = 'bd_cidades_estados'
user = ''
pwd = ''

In [ ]:
host = 'awspostgresql-lc.cpsrkdcyo3lf.us-east-2.rds.amazonaws.com'
dbName = 'geo_letscode'
user = 'postgres'
pwd = '.Sbrobous'

In [ ]:
# Funcões

# Função para conexão com o banco de dados
def connect_database(host, dbName, user, pwd):
    """
    Função para conectar o banco de dados

    Parametros:
        host   : Instancia PostgreSQL
        dbName : banco de dados
        user   : usuário
        pwd    : senha do usuário
        
    Retornos:
        cur  : Utilize para conectar e executar os comandos SQL.
        conn : Conexão com o banco de dados.
    """
    # connect to default database
    conn = psycopg2.connect(f"host={host} dbname={dbName} user={user} password={pwd}")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    return cur, conn

# Função para criar as tabelas
def create_tables(stmt_create_tables):
    """
    Função para criar tabelas

    Parametros:
        cur  : Utilize para conectar e executar os comandos SQL.
        conn : Conexão com o banco de dados.
    """
    for stmt in stmt_create_tables:
        cur.execute(stmt)
        conn.commit()
    return


# Função genérica para inserir linhas
def insert_rows(df, insert_stmt):
    """
    Função genérica para inserir linhas em tabela
    
    Parametros:
        df          : Dataframe com os valores a serem inseridos
        insert_stmt : Comando de insert 
    """
    # Executa o loop para gerar as tuplas com os valores a serem inseridos
    for row in df.itertuples(index=False):
        # print(tuple(row))
        # Executa o insert
        cur.execute(insert_stmt, tuple(row))
    print("Dados inseridos com sucesso...")
    return

# função usando psycopg2.extras.execute_values() para inserir o dataframe
def insert_tb_cidade(df):    
    """
    Função usando psycopg2.extras.execute_values() para inserir o dataframe
    
    Parametros:
        df    : Dataframe com os valores a serem inseridos
    """   
    # Criação de uma lista de tupples a partir dos valores do dataframe
    tpls = [tuple(row) for row in df.itertuples(index=False)]
    
    # Comando de inserção
    sql = """ INSERT INTO tb_cidade (nome_cidade, id_estado) VALUES %s"""
    try:
        extras.execute_values(cur, sql, tpls)
        print("Dados inseridos usando execute_values() com sucesso...")
    except (Exception, psycopg2.DatabaseError) as err:
        show_psycopg2_exception(err)
        cursor.close()
    return

# função usando copy_from() com StringIO para inserir o dataframe
def copy_from_dataFile_StringIO(df, table, cols):
    """
    Função usando copy_from() com StringIO para inserir o dataframe

    Parametros:
        df    : Dataframe com os valores a serem inseridos
        table : Tabela para inserção dos dados
        cols  : Colunas da tabela
    """
    
  # salvando o dataframe para a memória
    buffer = StringIO()
    df.to_csv(buffer, header=False, index = False)
    buffer.seek(0)
    try:
        cur.copy_from(buffer, table, sep=",", columns=cols)
        print("Dados inseridos usando copy_from_datafile_StringIO() com sucesso...")
    except (Exception, psycopg2.DatabaseError) as err:
        show_psycopg2_exception(err)
        cursor.close()
    return


In [ ]:
# Conexão com o banco de dados Postgre
cur, conn = connect_database(host, dbName, user, pwd)

In [ ]:
# Comandos para criação das tabelas
#stmt_create_tables = list()
#stmt_create_tables.append('CREATE TABLE IF NOT EXISTS fabricante (manufacturerID INTEGER NOT NULL, manufacturer VARCHAR(100) NOT NULL, CONSTRAINT fabricante_pk PRIMARY KEY (manufacturerID));')
#stmt_create_tables.append('CREATE TABLE IF NOT EXISTS produtos (productID INTEGER NOT NULL, category VARCHAR(100) NOT NULL, segment VARCHAR(100) NOT NULL, product VARCHAR(100) NOT NULL, manufacturerID INTEGER NOT NULL, CONSTRAINT produtos_pk PRIMARY KEY (productID));')
#stmt_create_tables.append('CREATE TABLE IF NOT EXISTS geo (zip INTEGER NOT NULL, city VARCHAR(100) NOT NULL, state VARCHAR(2) NOT NULL, region VARCHAR(6) NOT NULL, district VARCHAR(20) NOT NULL, CONSTRAINT geo_pk PRIMARY KEY (zip));')
#stmt_create_tables.append('CREATE TABLE IF NOT EXISTS vendas (saleid INTEGER NOT NULL, date DATE NOT NULL, units INTEGER NOT NULL, revenue NUMERIC(10,2) NOT NULL, zip INTEGER NOT NULL, productID INTEGER NOT NULL, CONSTRAINT vendas_pk PRIMARY KEY (saleid));')


In [ ]:
#cur.execute('ALTER TABLE produtos ADD CONSTRAINT fabricante_produtos_fk FOREIGN KEY (manufacturerID) REFERENCES fabricante (manufacturerID) ON DELETE NO ACTION ON UPDATE NO ACTION NOT DEFERRABLE;')  
#cur.execute('ALTER TABLE vendas ADD CONSTRAINT produtos_vendas_fk FOREIGN KEY (productID) REFERENCES produtos (productID) ON DELETE NO ACTION ON UPDATE NO ACTION NOT DEFERRABLE;')  
#cur.execute('ALTER TABLE vendas ADD CONSTRAINT geo_vendas_fk FOREIGN KEY (zip) REFERENCES geo (zip) ON DELETE NO ACTION ON UPDATE NO ACTION NOT DEFERRABLE;')  

In [ ]:
# Cria as tabelas no banco de dados
#create_tables(stmt_create_tables)

In [ ]:
# Cria datraframe geo
dfgeo = pd.read_csv('./bases/geo.csv', sep=',')

In [ ]:
dfgeo.shape

In [ ]:
# Verifica se existem zips duplicados
dfgeo[dfgeo.duplicated()]

In [ ]:
# Retira o State de City e cria uma coluna apenas com City
dfgeo['CityOnly'] = dfgeo['City'].str.split(',', 0, expand=True)[0]

In [ ]:
# Gera dataframe geo2
columns = ['Zip', 'CityOnly', 'State', 'Region', 'District']
dfgeo2 = dfgeo[columns].copy()
dfgeo2.rename(columns = {'CityOnly': 'City'}, inplace = True)

In [ ]:
# Inserindo dados na tabela geo
#Comando para truncar a tabela 
cur.execute('truncate table geo')  

#Chamada da função para inserção utilizando arquivo em memória
cols = ['zip', 'City', 'State', 'Region', 'District']
copy_from_dataFile_StringIO(dfgeo2, 'geo', cols)

In [ ]:
#Comando para truncar a tabela 
cur.execute('truncate table tb_estado restart identity')  

#Chamada da função para inserção utilizando arquivo em memória
cols = ['nome_estado', 'sigla_estado']
copy_from_dataFile_StringIO(df_estado, 'tb_estado', cols)

In [ ]:
# Select na tabela tb_estado para verificar as linhas inseridas
cur.execute('SELECT * FROM tb_estado')  
rows = cur.fetchall()
rows

In [ ]:
# Cria o Dataframe com os estados a partir da leitura da tabela para carregar o id_estado
df_estadoSQL = pd.read_sql('SELECT * FROM tb_estado', conn);

In [ ]:
df_estadoSQL

In [ ]:
# Cria o dataframe df_cidade com o join do dataframe df (planilha) e dataframe df_estadoSQL 
# para referenciar o id_estado nas cidades
df_cidade = pd.merge(left=df, right=df_estadoSQL, left_on='Sigla', right_on='sigla_estado').copy()

In [ ]:
# Processo importando o dataframe para a memória e depois para a tabela é mais performático
cur.execute('truncate table tb_cidade restart identity')

#definição das colunas da tabela
cols=['nome_cidade', 'id_estado']

#função de inserção
copy_from_dataFile_StringIO(df_cidade[['Cidade', 'id_estado']], 'tb_cidade', cols)

In [ ]:
# Select na tabela tb_cidade para contabilizar as cidades por estado
cur.execute("""
            select 	count(id_cidade) AS "Qtd_Cidades", 
                    e.nome_estado, 
                    e.sigla_estado 
            from 		tb_cidade c 
            inner join 	tb_estado e 
                    on	e.id_estado = c.id_estado  
            group by e.nome_estado, e.sigla_estado
            order by 1 asc
            """)
rows = cur.fetchall()
rows

In [ ]:
# Select na tabela tb_cidade para verificar as linhas inseridas
cur.execute('SELECT * FROM tb_cidade')  
rows = cur.fetchall()
rows

In [ ]:
##############################################################################################
#
# 2. VERSÃO
#
# NÃO É NECESSÁRIO EXECUTAR ESSA CÉLULA, ESTE INSERT É MENOS PERFORMATICO QUE O COPY_FROM E StringIO
# MAS MUITO MELHOR QUE O INSERT LINHA A LINHA
# 
# EXPERIÊNCIA DIDÁTICA
#
##############################################################################################

# Processo incluindo em lote utilizando psycopg2.extras.execute_values() é mais performático
cur.execute('truncate table tb_cidade restart identity')  
insert_tb_cidade(df_cidade[['Cidade', 'id_estado']])

In [ ]:
##############################################################################################
#
# 1. VERSÃO
#
# NÃO É NECESSÁRIO EXECUTAR ESSA CÉLULA, POIS O INSERT LINHA A LINHA É DEMORADO
#
# EXPERIÊNCIA DIDÁTICA
#
##############################################################################################

# Processo incluindo linha a linha - é bem demorado (15 min = 5.000)
postgres_insert_query = """ INSERT INTO tb_cidade (id_cidade, nome_cidade, id_estado) VALUES (DEFAULT,%s,%s)"""

cur.execute('truncate table tb_cidade restart identity')  
insert_rows(df_cidade[['Cidade', 'id_estado']], postgres_insert_query)